In [1]:
import pandas as pd
import yaml
import os
import sys
from dateutil.relativedelta import relativedelta
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
import re
import numpy as np

bucket = S3BucketUtils()

from dict_of_vars import variable_map
import read_a_combination_of_variables

def add_month(date, m):
    ddd = pd.to_datetime(date, format='%Y-%m-%d')
    ddd2 = ddd + relativedelta(months=m)
    return (str(ddd2))[0:10]

In [2]:
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    model_params = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = model_params['date_of_analysis']

date_dir = date_of_analysis.replace('-', '_')

In [3]:
previous_date_of_analysis = add_month(date_of_analysis, -1)

In [4]:
date_dir

'2023_02_01'

In [5]:
df = pd.read_csv('data/' + date_dir + 
                 '/exports/important_variables/important_variables_sorted_by_variable_name.csv')

In [6]:
all_spots_sets = ['ALL_spots_with_CB_canc_req',\
                     'CAN_CANCEL_spots_wo_CB_canc_req']

In [7]:
df_ = bucket.load_csv_from_s3(file_name = 'churn_analysis_based_on_behaviour/data/' + date_dir + \
                    '/exports/data_used_for_each_model/data_tv_'+all_spots_sets[0]+'.csv')
df_c = bucket.load_csv_from_s3(file_name = 'churn_analysis_based_on_behaviour/data/' + date_dir + \
                    '/exports/data_used_for_each_model/data_tv_'+all_spots_sets[1]+'.csv')
df_model_names = bucket.load_csv_from_s3(\
file_name = 'churn_analysis_based_on_behaviour/combinations_of_variables_that_are_not_dependent/model_names.csv')

In [8]:
df['Percent_Not_Doing_ALL'] = 100 - df['last_month_%_of_ALL_spots_with_CB_canc_req']
df['Percent_Not_Doing_CAN_CANCEL'] = 100 - df['last_month_%_of_CAN_CANCEL_spots_wo_CB_canc_req']

In [9]:
df[df['Percent_Not_Doing_ALL'].isnull()]

,model,variable,interpretation,important_variables_group,exp(coef) - AVERAGE,p value - AVERAGE,ALL_spots_with_CB_cancellation_requested_exp(coef)_and_p_value,CAN_CANCEL_spots_wo_CB_cancellation_requested_exp(coef)_and_p_value,%_of_ALL_spots_with_CB_canc_req,last_month_%_of_ALL_spots_with_CB_canc_req,%_of_CAN_CANCEL_spots_wo_CB_canc_req,last_month_%_of_CAN_CANCEL_spots_wo_CB_canc_req,Percent_Not_Doing_ALL,Percent_Not_Doing_CAN_CANCEL
1,model_started,Fb.page.likes.analysis.avg.log2,NaN,significant_vars,0.98,0.005,"(0.9762, 0.005)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,model_started,New.email.subscribers.allinclusive.last.month....,NaN,significant_vars,0.95,0.003,"(0.9532, 0.003)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,model_started,Website.Views.last.month.total.log2,NaN,significant_vars,0.81,0.000,"(0.8195, 0.0)","(0.808, 0.0)",NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df[df['Percent_Not_Doing_CAN_CANCEL'].isnull()]

,model,variable,interpretation,important_variables_group,exp(coef) - AVERAGE,p value - AVERAGE,ALL_spots_with_CB_cancellation_requested_exp(coef)_and_p_value,CAN_CANCEL_spots_wo_CB_cancellation_requested_exp(coef)_and_p_value,%_of_ALL_spots_with_CB_canc_req,last_month_%_of_ALL_spots_with_CB_canc_req,%_of_CAN_CANCEL_spots_wo_CB_canc_req,last_month_%_of_CAN_CANCEL_spots_wo_CB_canc_req,Percent_Not_Doing_ALL,Percent_Not_Doing_CAN_CANCEL
1,model_started,Fb.page.likes.analysis.avg.log2,NaN,significant_vars,0.98,0.005,"(0.9762, 0.005)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,model_started,New.email.subscribers.allinclusive.last.month....,NaN,significant_vars,0.95,0.003,"(0.9532, 0.003)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,model_started,Website.Views.last.month.total.log2,NaN,significant_vars,0.81,0.000,"(0.8195, 0.0)","(0.808, 0.0)",NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
model_numbers = \
read_a_combination_of_variables.\
get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [12]:
len(set(model_numbers))

46

In [13]:
variables_coefficients = []

for i in [n for n in model_numbers if n!=0]:
    path = 'data/' + date_dir + \
    '/exports/coefficients_and_pvalues/started_doing_something/model_'+str(i) + \
    '/coef_and_pvalues_ALL_spots_with_CB_cancellation_requested_p_below_0_2.csv'
    df_mod = pd.read_csv(path)
    for k in df_mod.covariate.tolist():
        variables_coefficients.append({'var': k, 'model_no': i, 'model': 'started', 'set':'all', \
                                       'c_exp': df_mod.loc[df_mod.covariate==k]['exp(coef)'].iloc[0], \
                                       'p': df_mod.loc[df_mod.covariate==k]['p'].iloc[0]})
    path = 'data/' + date_dir + \
    '/exports/coefficients_and_pvalues/started_doing_something/model_'+str(i) + \
    '/coef_and_pvalues_CAN_CANCEL_spots_wo_CB_cancellation_requested_p_below_0_2.csv'
    df_mod = pd.read_csv(path)
    for k in df_mod.covariate.tolist():
        variables_coefficients.append({'var': k, 'model_no': i, 'model': 'started', 'set':'can_cancel', \
                                       'c_exp': df_mod.loc[df_mod.covariate==k]['exp(coef)'].iloc[0], \
                                       'p': df_mod.loc[df_mod.covariate==k]['p'].iloc[0]})

In [14]:
df_coefs = pd.DataFrame(variables_coefficients)

In [15]:
df_coefs['base_name'] = np.NaN

In [16]:
df_coefs.loc[df_coefs['var'].map(lambda x: True if re.search('had_', x) else False), 'base_name'] = \
df_coefs.loc[df_coefs['var'].map(lambda x: True if re.search('had_', x) else False), 'var']\
.map(lambda x: x[4:-31] if re.search('didnt_last_', x) else x[4:-14])

In [17]:
df_coefs_max = df_coefs[df_coefs['base_name'].notnull()]

In [18]:
df['base_name'] = np.NaN
df.loc[df['variable'].map(lambda x: True if re.search('had_', x) else False), 'base_name'] = \
df.loc[df['variable'].map(lambda x: True if re.search('had_', x) else False), 'variable']\
.map(lambda x: x[4:-31] if re.search('didnt_last_', x) else x[4:-14])

In [19]:
df[df['base_name'].notnull()]['base_name'].nunique()

27

In [20]:
df_coefs_final = \
df_coefs_max[df_coefs_max['base_name'].isin(df[df['base_name'].notnull()]['base_name'].unique())]

In [21]:
def find_minimal_p_values_all(df_):
    if 'all' in df_['set'].unique():
        # df__ = df_[df_['set'].isin(['all'])].groupby(['set', 'model_no'])['p'].sum().reset_index()
        df__ = df_[df_['set'].isin(['all'])].groupby(['set', 'model_no'])['p'].apply(lambda x: x.tolist()[1]).reset_index()
        df__ = df__[df__['p']==df__['p'].min()]
        df__ = list(zip(df__['set'], df__['model_no']))
    else:
        df__ = np.NaN
    return(df__)

In [22]:
def find_minimal_p_values_can_cancel(df_):
    if 'can_cancel' in df_['set'].unique():
        # df__ = df_[df_['set'].isin(['can_cancel'])].groupby(['set', 'model_no'])['p'].sum().reset_index()
        df__ = df_[df_['set'].isin(['can_cancel'])].groupby(['set', 'model_no'])['p'].apply(lambda x: x.tolist()[1]).reset_index()
        df__ = df__[df__['p']==df__['p'].min()]
        df__ = list(zip(df__['set'], df__['model_no']))
    else:
        df__ = np.NaN
    return(df__)

In [23]:
def get_last_month_data(df, last_month_date, customers=False, cancelled=False):
    
    df_ = df.copy()
    df_ = df_[df_['right_limit']==last_month_date]
    
    if customers:
        df_ = df_[df_['event']==False]
    if cancelled:
        df_ = df_[df_['event']==True]
    
    return(df_)

In [24]:
def get_stopped_last_month_data(df, last_month_date, var_had_and_didnt, customers=False, cancelled=False):
    
    df_ = get_last_month_data(df, last_month_date, customers, cancelled)
    
    df_ = df_[(df_[var_had_and_didnt]\
               .map(lambda x: x==1))]
    return(df_)

In [25]:
def get_never_did_last_month_data(df, last_month_date, var_had_and_didnt, var_had_last, customers=False, cancelled=False):
    
    df_ = get_last_month_data(df, last_month_date, customers, cancelled)
    
    df_ = df_[(df_[[var_had_and_didnt, var_had_last]]\
               .apply(lambda x: x[0]==0 and x[1]==0, axis=1))]
    return(df_)

In [26]:
def get_started_last_month_data(df, last_month_date, var_had_last, period, customers=False, cancelled=False):
    
    df_ = get_last_month_data(df, last_month_date, customers, cancelled)
    
    df_ = df_[df_[var_had_last].map(lambda x: x==1)]
    
    df_ = df_[df_['time'] <= period]
    
    return(df_)

In [27]:
def get_continued_last_month_data(df, last_month_date, var_had_last, period, customers=False, cancelled=False):
    
    df_ = get_last_month_data(df, last_month_date, customers, cancelled)
    
    df_ = df_[df_[var_had_last].map(lambda x: x==1)]
    
    df_ = df_[df_['time'] > period]
    
    return(df_)

In [28]:
def get_variable_model_by_name(var_):
    df_ = df_model_names[df_model_names['model_name']==var_]
    if df_.shape[0]>0:
        model_no_ = df_['model_number'].iloc[0]
    else:
        model_no_ = np.NaN
    return(model_no_)

In [29]:
# df_max_impact = pd.DataFrame()

# df_max_impact['variable'] = df[df['base_name'].notnull()]['base_name'].unique()

# df_max_impact = \
# df_max_impact.merge(df_coefs_final.groupby('base_name')['var']\
#                     .apply(lambda x: x.drop_duplicates().sort_values().tolist())\
#                     .reset_index().rename(columns={'base_name':'variable', 'var':'vars'}), \
#                     on='variable', how='left')

df_max_impact = df_coefs[df_coefs['base_name'].notnull()].groupby(['model_no', 'set', 'base_name'])\
['var'].apply(list).reset_index()[['base_name', 'var']].drop_duplicates(subset=['base_name'])\
.rename(columns={'base_name':'variable', 'var':'vars'})

df_max_impact['months'] = \
df_max_impact['vars'].map(lambda x: x[1][-8:])

df_max_impact.loc[(df_max_impact['variable']=='number_of_requests_for_new_text_fragment_'), 'months'] = \
'4_months'

df_max_impact['months'] = \
df_max_impact['months'].map(lambda x: int(x[:1]))


df_max_impact['models_all_for_coefs'] = df_max_impact['variable']\
.apply(lambda x: find_minimal_p_values_all(df_coefs[df_coefs['base_name'].isin([x])]))


df_max_impact['models_can_cancel_for_coefs'] = df_max_impact['variable']\
.apply(lambda x: find_minimal_p_values_can_cancel(df_coefs[df_coefs['base_name'].isin([x])]))

df_max_impact['models_by_varname'] = df_max_impact['variable']\
.map(lambda x: get_variable_model_by_name(x))


df_max_impact['spots_never_did_customers'] = \
df_max_impact['vars']\
.map(lambda x: get_never_did_last_month_data(df_, previous_date_of_analysis, 
                                             var_had_and_didnt=x[0], var_had_last=x[1], 
                                             customers=True).shape[0])


df_max_impact['spots_customers'] = \
get_last_month_data(df_, previous_date_of_analysis, customers=True).shape[0]


df_max_impact['Not_Doing'] = df_max_impact['spots_never_did_customers']
df_max_impact['Percent_Not_Doing'] = df_max_impact['Not_Doing']/df_max_impact['spots_customers']


df_max_impact['Continue'] = \
df_max_impact[['vars', 'months']]\
.apply(lambda x: get_continued_last_month_data(df_, previous_date_of_analysis, var_had_last=x['vars'][1], 
                                             period=x['months'], customers=True).shape[0], axis=1)

df_max_impact['Stopped_Doing'] = \
df_max_impact['vars']\
.map(lambda x: get_stopped_last_month_data(df_, previous_date_of_analysis, var_had_and_didnt=x[0], 
                                                customers=True).shape[0])

df_max_impact['Percent_Who_Quit'] = df_max_impact['Stopped_Doing']/(df_max_impact['Continue']+df_max_impact['Stopped_Doing'])

In [30]:
df_max_impact

,variable,vars,months,models_all_for_coefs,models_can_cancel_for_coefs,models_by_varname,spots_never_did_customers,spots_customers,Not_Doing,Percent_Not_Doing,Continue,Stopped_Doing,Percent_Who_Quit
0,added_events_manually_edited_events,[had_added_events_manually_edited_events_befor...,4,"[(all, 33)]","[(can_cancel, 24)]",22.0,3732,4932,3732,0.756691,763,340,0.308250
1,clicked_emails,[had_clicked_emails_before_and_didnt_last_4_mo...,4,"[(all, 18)]","[(can_cancel, 19)]",19.0,4015,4932,4015,0.814071,239,655,0.732662
2,downloaded_qrcode_flyers,[had_downloaded_qrcode_flyers_before_and_didnt...,3,"[(all, 14)]",NaN,33.0,3850,4932,3850,0.780616,241,778,0.763494
3,emails_sent_manually_scheduled_emails_sent,[had_emails_sent_manually_scheduled_emails_sen...,3,"[(all, 7)]","[(can_cancel, 5)]",25.0,2300,4932,2300,0.466342,1892,634,0.250990
4,feedback_submissions,[had_feedback_submissions_before_and_didnt_las...,2,"[(all, 18)]",NaN,NaN,4663,4932,4663,0.945458,111,152,0.577947
5,incentive_downloads,[had_incentive_downloads_before_and_didnt_last...,2,"[(all, 26)]","[(can_cancel, 26)]",NaN,4860,4932,4860,0.985401,32,40,0.555556
6,posts_on_facebook,[had_posts_on_facebook_before_and_didnt_last_2...,2,"[(all, 8)]","[(can_cancel, 1)]",1.0,2311,4932,2311,0.468573,1396,1125,0.446251
7,preview_page_views_email,[had_preview_page_views_email_before_and_didnt...,4,"[(all, 44)]","[(can_cancel, 24)]",8.0,2335,4932,2335,0.473439,1438,906,0.386519
8,preview_page_views_twitter,[had_preview_page_views_twitter_before_and_did...,4,"[(all, 19)]","[(can_cancel, 19)]",9.0,3732,4932,3732,0.756691,475,635,0.572072
9,private_parties_submissions,[had_private_parties_submissions_before_and_di...,3,"[(all, 12)]","[(can_cancel, 12)]",12.0,2713,4932,2713,0.550081,1908,256,0.118299


In [31]:
df_all = df_max_impact[['variable', 'vars', 'models_all_for_coefs']]\
.explode('models_all_for_coefs')

In [32]:
df_cc = df_max_impact[['variable', 'vars', 'models_can_cancel_for_coefs']]\
.explode('models_can_cancel_for_coefs')

In [33]:
df_all['p'] = df_all\
.apply(lambda x: df_coefs[(df_coefs['set']==x['models_all_for_coefs'][0]) & \
                          (df_coefs['model_no']==x['models_all_for_coefs'][1]) & \
                          (df_coefs['var'].isin(x['vars']))] \
       if type(x['models_all_for_coefs'])==tuple else x['models_all_for_coefs'], axis=1)\
.map(lambda x: tuple(zip(x['var'], x['p'])) if type(x)==pd.DataFrame else x)

In [34]:
df_all['c_exp'] = df_all\
.apply(lambda x: df_coefs[(df_coefs['set']==x['models_all_for_coefs'][0]) & \
                          (df_coefs['model_no']==x['models_all_for_coefs'][1]) & \
                          (df_coefs['var'].isin(x['vars']))] \
       if type(x['models_all_for_coefs'])==tuple else x['models_all_for_coefs'], axis=1)\
.map(lambda x: tuple(zip(x['var'], x['c_exp'])) if type(x)==pd.DataFrame else x)

In [35]:
df_cc['p'] = df_cc\
.apply(lambda x: df_coefs[(df_coefs['set']==x['models_can_cancel_for_coefs'][0]) & \
                          (df_coefs['model_no']==x['models_can_cancel_for_coefs'][1]) & \
                          (df_coefs['var'].isin(x['vars']))] \
       if type(x['models_can_cancel_for_coefs'])==tuple else x['models_can_cancel_for_coefs'], axis=1)\
.map(lambda x: tuple(zip(x['var'], x['p'])) if type(x)==pd.DataFrame else x)

In [36]:
df_cc['c_exp'] = df_cc\
.apply(lambda x: df_coefs[(df_coefs['set']==x['models_can_cancel_for_coefs'][0]) & \
                          (df_coefs['model_no']==x['models_can_cancel_for_coefs'][1]) & \
                          (df_coefs['var'].isin(x['vars']))] \
       if type(x['models_can_cancel_for_coefs'])==tuple else x['models_can_cancel_for_coefs'], axis=1)\
.map(lambda x: tuple(zip(x['var'], x['c_exp'])) if type(x)==pd.DataFrame else x)

In [37]:
df_all.p.map(lambda x: [i[1] for i in x] if type(x)==tuple else x).iloc[0]

[0.1800996347267912, 0.5186878353575664]

In [38]:
df_all.p.map(lambda x: [i[0] for i in x] if type(x)==tuple else x).iloc[0]

['had_added_events_manually_edited_events_before_and_didnt_last_4_months',
 'had_added_events_manually_edited_events_last_4_months']

In [39]:
df_all.p.map(lambda x: [i[1] for i in x] if type(x)==tuple else x).iloc[1]

[0.0137558142899026, 0.0029192664489068]

In [40]:
df_all.p.map(lambda x: [i[0] for i in x] if type(x)==tuple else x).iloc[1]

['had_clicked_emails_before_and_didnt_last_4_months',
 'had_clicked_emails_last_4_months']

In [41]:
df_max_impact['c_exp_mean_all'] = \
df_max_impact['variable']\
.map(lambda x: np.mean(df_all[df_all['variable']==x]['c_exp'].map(lambda x: x[1][1] if type(x)==tuple else x).tolist()))

In [42]:
df_max_impact['p_exp_mean_all'] = \
df_max_impact['variable']\
.map(lambda x: np.mean(df_all[df_all['variable']==x]['p'].map(lambda x: x[1][1] if type(x)==tuple else x).tolist()))

In [43]:
df_max_impact['c_exp_mean_can_cancel'] = \
df_max_impact['variable']\
.map(lambda x: np.mean(df_cc[df_cc['variable']==x]['c_exp'].map(lambda x: x[1][1] if type(x)==tuple else x).tolist()))

In [44]:
df_max_impact['p_exp_mean_can_cancel'] = \
df_max_impact['variable']\
.map(lambda x: np.mean(df_cc[df_cc['variable']==x]['p'].map(lambda x: x[1][1] if type(x)==tuple else x).tolist()))

In [45]:
df_max_impact.columns

Index(['variable', 'vars', 'months', 'models_all_for_coefs',
       'models_can_cancel_for_coefs', 'models_by_varname',
       'spots_never_did_customers', 'spots_customers', 'Not_Doing',
       'Percent_Not_Doing', 'Continue', 'Stopped_Doing', 'Percent_Who_Quit',
       'c_exp_mean_all', 'p_exp_mean_all', 'c_exp_mean_can_cancel',
       'p_exp_mean_can_cancel'],
      dtype='object')

In [46]:
def choose_coefficient(row_):
    c_all = row_['c_exp_mean_all']
    p_all = row_['p_exp_mean_all']
    c_cc = row_['c_exp_mean_can_cancel']
    p_cc = row_['p_exp_mean_can_cancel']
    if str(row_['p_exp_mean_all'])=='nan':
        return(c_cc)
    if str(row_['p_exp_mean_can_cancel'])=='nan':
        return(c_all)
    if p_all < p_cc:
        return(c_all)
    else:
        return(c_cc)

In [47]:
def p_of_choosen_coefficient(row_):
    c_all = row_['c_exp_mean_all']
    p_all = row_['p_exp_mean_all']
    c_cc = row_['c_exp_mean_can_cancel']
    p_cc = row_['p_exp_mean_can_cancel']
    if str(row_['p_exp_mean_all'])=='nan':
        return(p_cc)
    if str(row_['p_exp_mean_can_cancel'])=='nan':
        return(p_all)
    if p_all < p_cc:
        return(p_all)
    else:
        return(p_cc)

In [48]:
df_max_impact['c_exp_choosen'] = \
df_max_impact[['c_exp_mean_all', 'p_exp_mean_all', 
               'c_exp_mean_can_cancel', 'p_exp_mean_can_cancel']]\
.apply(lambda x: choose_coefficient(x), axis=1)

df_max_impact['p_of_c_exp_choosen'] = \
df_max_impact[['c_exp_mean_all', 'p_exp_mean_all', 
               'c_exp_mean_can_cancel', 'p_exp_mean_can_cancel']]\
.apply(lambda x: p_of_choosen_coefficient(x), axis=1)

df_max_impact['Coef_Continue'] = df_max_impact['c_exp_choosen']

df_max_impact['Continue_Churn_Improvement'] = (1-df_max_impact['Coef_Continue']) * \
df_max_impact['Percent_Not_Doing'] * (1-df_max_impact['Percent_Who_Quit'])

In [49]:
df_max_impact = df_max_impact.sort_values('variable')

In [50]:
df_max_impact.loc[df_max_impact['p_of_c_exp_choosen']>.2, 'Continue_Churn_Improvement'] = np.NaN

In [51]:
df_max_impact.to_csv('max_churn_improvement_all_model_vars.csv', index=False)

In [67]:
df_max_impact[df_max_impact['variable']=='visited_admin'].iloc[0]

variable                                                           visited_admin
vars                           [had_visited_admin_before_and_didnt_last_2_mon...
months                                                                         2
models_all_for_coefs                                                 [(all, 26)]
models_can_cancel_for_coefs    [(can_cancel, 6), (can_cancel, 42), (can_cance...
models_by_varname                                                           26.0
spots_never_did_customers                                                    401
spots_customers                                                             4932
Not_Doing                                                                    401
Percent_Not_Doing                                                       0.081306
Continue                                                                    3437
Stopped_Doing                                                                651
Percent_Who_Quit            

In [61]:
df_max_impact[df_max_impact['variable']=='visited_admin']['vars'].iloc[0]

['had_visited_admin_before_and_didnt_last_2_months',
 'had_visited_admin_last_2_months']

In [62]:
previous_date_of_analysis

'2023-01-01'

In [64]:
get_never_did_last_month_data(df_, previous_date_of_analysis, \
                              var_had_and_didnt='had_visited_admin_before_and_didnt_last_2_months', \
                              var_had_last='had_visited_admin_last_2_months', \
                              customers=True).shape

(401, 403)

In [85]:
get_last_month_data(df_, previous_date_of_analysis, customers=True).shape

(4932, 403)

In [69]:
get_continued_last_month_data(df_, previous_date_of_analysis, 
                              var_had_last='had_visited_admin_last_2_months', 
                              period=2, customers=True).shape

(3437, 403)

In [90]:
spots_nd = \
get_never_did_last_month_data(df_, previous_date_of_analysis, \
                              var_had_and_didnt='had_visited_admin_before_and_didnt_last_2_months', \
                              var_had_last='had_visited_admin_last_2_months', \
                              customers=True).spot_id.unique()
spots_c = \
get_continued_last_month_data(df_, previous_date_of_analysis, 
                              var_had_last='had_visited_admin_last_2_months', 
                              period=2, customers=True).spot_id.unique()

spots_all = \
get_last_month_data(df_, previous_date_of_analysis, customers=True).spot_id.unique()

spots_s = \
get_stopped_last_month_data(df_, previous_date_of_analysis, var_had_and_didnt='had_visited_admin_before_and_didnt_last_2_months', 
                            customers=True).spot_id.unique()

In [87]:
len(spots_all)

4932

In [91]:
len(spots_c) + len(spots_nd) + len(spots_s)

4489

In [92]:
df_all_cust = get_last_month_data(df_, previous_date_of_analysis, customers=True)

In [96]:
df_all_cust[~(df_all_cust['spot_id'].isin(spots_c.tolist()+spots_nd.tolist()+spots_s.tolist()))]\
.start.value_counts()

2022-11-01    259
2022-12-01    184
Name: start, dtype: int64